## Import Liberares

In [4]:
import cv2
import numpy as np
import mediapipe as mp
from scipy.signal import butter, filtfilt
from mediapipe.python.solutions.pose import POSE_CONNECTIONS

## Butterworth filter functions

Here we use **Butterworth filters** via Scipy library. [signal.butter](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html) is a function provided by the [scipy.signal](https://docs.scipy.org/doc/scipy/reference/signal.html) module in Python. It is used to design digital Butterworth filters, which are a type of infinite impulse response (IIR) filter commonly used for signal processing tasks, including filtering EEG signals. The Butterworth filter is characterized by having a flat frequency response in its passband and a gradual roll-off in its stopband.

In [1]:
def butter_lowpass(cutoff, fs, order=2):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq   # normalized cutoff frequency
    b, a = butter(order, normal_cutoff, btype = "low", analog = False)
    return b, a


def butter_lowpass_filter(data, cutoff, fs, order=2):
    b, a = butter_lowpass(cutoff, fs, order=order)
    return filtfilt(b, a, data)
    

## 4. Load MediaPipe Pose Model


In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

cap = cv2.VideoCapture("pose.mp4")

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0 or np.isnan(fps):
    fps = 30

width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

out = cv2.VideoWriter("pose-result.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))